In [11]:
import requests
import pandas as pd

API_KEY = 'CG-ZGHLZY66rLn27kHpEeRLC4EC'
filtered_out_ids = ['weth', 'vnst-stablecoin', 'ripple', 'wrapped-solana', 'ether-fi', 
                    'wrapped-bitcoin', 'binance-bridged-usdt-bnb-smart-chain', 'wbnb', 'jupiter-exchange-solana']

import time

def get_coin_data(api_url, retry_count=3, wait_time=60):
    headers = {'x-cg-demo-api-key': API_KEY}
    
    for attempt in range(retry_count):
        try:
            response = requests.get(api_url, headers=headers)
            response.raise_for_status()  # Raise an exception for 4xx or 5xx status codes
            data = response.json()
            return data
        except requests.exceptions.RequestException as e:
            if response.status_code == 429:
                if attempt < retry_count - 1:
                    print(f"Request throttled {api_url}. Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    print("Max retry attempts reached. Skipping this request.")
            else:
                print(f"Error occurred: {e}")
                print(f"Response status code: {response.status_code}")
                print(f"Response text: {response.text}")
                break
    
    return None


url_24h = 'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=volume_desc&per_page=75&page=1&sparkline=false'
coins_24h = pd.DataFrame(get_coin_data(url_24h))

url_stablecoins = 'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&category=stablecoins'
stablecoins = pd.DataFrame(get_coin_data(url_stablecoins))
stablecoin_ids = stablecoins['id'].tolist()

coins_filtered = coins_24h[~coins_24h['id'].isin(stablecoin_ids + filtered_out_ids)]
coins_filtered['total_volume'] = pd.to_numeric(coins_filtered['total_volume'], errors='coerce')
coins_filtered = coins_filtered.sort_values(by='total_volume', ascending=False).head(40)

columns_to_drop = ['image', 'fully_diluted_valuation', 'high_24h', 'low_24h', 'price_change_24h',
                   'price_change_percentage_24h', 'market_cap_change_24h', 'market_cap_change_percentage_24h',
                   'circulating_supply', 'total_supply', 'max_supply', 'ath', 'ath_change_percentage',
                   'ath_date', 'atl', 'atl_change_percentage', 'atl_date', 'roi', 'last_updated']


coins_filtered = coins_filtered.drop(columns=columns_to_drop)

print(coins_filtered.head(15))


                  id symbol           name  current_price    market_cap  \
1            bitcoin    btc        Bitcoin   70705.000000  1.394765e+12   
2           ethereum    eth       Ethereum    3644.870000  4.384191e+11   
4             solana    sol         Solana     176.200000  7.869628e+10   
6           dogecoin   doge       Dogecoin       0.197444  2.857884e+10   
9        binancecoin    bnb            BNB     578.720000  8.947188e+10   
10            ethena    ena         Ethena       1.360000  1.960530e+09   
11  the-open-network    ton        Toncoin       6.890000  2.401848e+10   
12          litecoin    ltc       Litecoin     100.510000  7.522867e+09   
13      bitcoin-cash    bch   Bitcoin Cash     670.430000  1.327761e+10   
14         shiba-inu   shib      Shiba Inu       0.000029  1.718985e+10   
15            tensor   tnsr         Tensor       1.990000  0.000000e+00   
16              near   near  NEAR Protocol       7.350000  7.958539e+09   
17              pepe   pe

/var/folders/dk/n5gn7cgn1_ddg9435cgt0r6r0000gn/T/ipykernel_99758/3041916366.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  coins_filtered['total_volume'] = pd.to_numeric(coins_filtered['total_volume'], errors='coerce')


In [2]:
# Fetch BTC price and 50-day SMA
url_btc_history = 'https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=50&interval=daily'
btc_history = get_coin_data(url_btc_history)
btc_prices = [price[1] for price in btc_history['prices']]
btc_sma50 = pd.Series(btc_prices).rolling(window=50).mean().iloc[-1]
btc_price = coins_filtered[coins_filtered['symbol'] == 'btc']['current_price'].values[0]

if btc_price > btc_sma50:
    print("BTC price is above the 50-day SMA. Favorable for long positions.")
else:
    print("BTC price is below the 50-day SMA. Not favorable for long positions.")

print(f"BTC current price: {btc_price:.2f}")
print(f"BTC 50-day SMA: {btc_sma50:.2f}")

BTC price is above the 50-day SMA. Favorable for long positions.
BTC current price: 70660.00
BTC 50-day SMA: 65271.71


In [3]:
coin_histories = {}
for index, row in coins_filtered.iterrows():
    coin_id = row['id']
    url_coin_history = f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart?vs_currency=usd&days=25&interval=daily'
    coin_history = get_coin_data(url_coin_history)
    coin_histories[coin_id] = coin_history
    try:
        coin_history = get_coin_data(url_coin_history)
        coin_histories[coin_id] = coin_history
    except Exception as e:
        print(f"Error occurred for {coin_id}: {str(e)}. Skipping this coin.")
        continue

Request throttled https://api.coingecko.com/api/v3/coins/filecoin/market_chart?vs_currency=usd&days=25&interval=daily. Retrying in 60 seconds...
Insufficient data for ethena. Removing from analysis.
Insufficient data for tensor. Removing from analysis.
Insufficient data for wormhole. Removing from analysis.


In [7]:
# Remove coins with insufficient price data from coin_histories
filtered_coin_histories = {}
for coin_id, coin_history in coin_histories.items():
    if 'prices' in coin_history and len(coin_history['prices']) >= 20:
        filtered_coin_histories[coin_id] = coin_history
    else:
        print(f"Insufficient data for {coin_id}. Removing from analysis.")

# Update the coin_histories dictionary with the filtered data
coin_histories = filtered_coin_histories

In [8]:
recommended_coins = []

# Calculate MA and check for cross within the last 10 days
for coin_id, coin_history in coin_histories.items():
    prices = [price[1] for price in coin_history['prices']]
    ma20 = [sum(prices[i-20:i]) / 20 for i in range(20, len(prices))]
    current_price = coins_filtered.loc[coins_filtered['id'] == coin_id, 'current_price'].values[0]
    
    cross_detected = False
    ma20_last_6_days = [sum(prices[i-20:i]) / 20 for i in range(20, len(prices))]
    prices_last_6_days = prices[-6:]
    
    for i in range(0, len(prices_last_6_days)):
        if (prices[i] > ma20_last_6_days[i]):
            cross_detected = True
            break

    if current_price > ma20[-1]:
        recommended_coins.append({'id': coin_id, 'current_price': current_price, 'ma20': ma20[-1], 'cross_detected': cross_detected})

# Create a DataFrame from the recommended_coins list
recommended_coins_df = pd.DataFrame(recommended_coins)

# Add the '24h_trading_volume' column to the DataFrame
recommended_coins_df['24h_trading_volume'] = recommended_coins_df['id'].apply(lambda x: coins_filtered.loc[coins_filtered['id'] == x, 'total_volume'].values[0])

# Rename columns for better readability
recommended_coins_df = recommended_coins_df.rename(columns={'id': 'Coin', 'current_price': 'Current Price', 'ma20': 'MA20', 'cross_detected': 'Cross Detected'})

recommended_coins_df

,Coin,Current Price,MA20,Cross Detected,24h_trading_volume
0,bitcoin,70660.000000,68369.700084,True,40205205178
1,ethereum,3642.540000,3465.870103,True,21059521092
2,dogecoin,0.198036,0.185411,False,2207190328
3,binancecoin,581.560000,577.436828,True,1729493575
4,the-open-network,6.860000,5.066955,False,833932363
5,litecoin,101.010000,96.361801,False,809689551
6,bitcoin-cash,671.400000,572.007762,False,754823537
7,shiba-inu,0.000029,0.000029,True,721583740
8,near,7.540000,6.952447,True,703478404
9,dogwifcoin,3.990000,3.390549,False,568934284


In [13]:
opened_position_coins = ['echelon-prime', 'ethereum', 'bitcoin', 'near', 
                         'sidus', 'dogecoin', 'yield-guild-games', 'litecoin', 'tron', 'neo', 'gala']

import pandas as pd

def should_sell(prices, ma20):
    if prices[-1] < ma20[-1]:
        return "Sell"
    else:
        return "Hold"

coin_data = []

for coin_id in opened_position_coins:
    url_coin_history = f'https://api.coingecko.com/api/v3/coins/{coin_id}/market_chart?vs_currency=usd&days=20&interval=daily'
    
    try:
        coin_history = get_coin_data(url_coin_history)
        
        prices = [price[1] for price in coin_history['prices']]
        if len(prices) >= 20:
            ma20 = [sum(prices[i-20:i]) / 20 for i in range(20, len(prices))]
            volume = coin_history['total_volumes'][-1][-1]
            coin_data.append({
                'Coin': coin_id,
                'Price': prices[-1],
                'MA20': ma20[-1],
                'Action': should_sell(prices, ma20),
                'Volume_24h': volume
            })
        else:
            print(f"Insufficient data for {coin_id}. Skipping this coin.")
    except Exception as e:
        print(f"Error occurred for {coin_id}: {str(e)}. Skipping this coin.")

df = pd.DataFrame(coin_data).sort_values(by='Volume_24h', ascending=False)
df['Volume_24h'] = df['Volume_24h'].apply(lambda x: f"{x:,.2f}")
df.reset_index(drop=True, inplace=True)
df.index += 1

print(df)

                 Coin         Price          MA20 Action         Volume_24h
1             bitcoin  70395.952003  68369.700084   Hold  34,676,517,320.11
2            ethereum   3644.157617   3465.870103   Hold  21,317,794,030.96
3            dogecoin      0.197298      0.185411   Hold   2,128,423,679.72
4         binancecoin    575.058324    577.436828   Sell   1,742,966,400.44
5            litecoin    100.071001     96.361801   Hold     797,351,373.21
6                near      7.282754      6.952447   Hold     694,053,339.45
7                 neo     21.062512     15.698512   Hold     468,400,165.32
8                tron      0.122264      0.119964   Hold     372,246,010.67
9   yield-guild-games      1.340039      1.220380   Hold     210,546,375.29
10      echelon-prime     24.825191     23.034927   Hold      12,448,585.19
11              sidus      0.012077      0.010759   Hold       1,778,940.45
